In [ ]:
# All the packages that were used 
import os
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
from random import sample
from sklearn.metrics import mean_squared_error



In [ ]:

# Please change this directory name if required
dir_name = "D:\\processed"
X = []
y = []
Y = []
files = os.listdir(dir_name)
count = 0
for file in files:
    # Only take .csv files from the directory
    if file.endswith(".csv"):
        count = count+1
        print(file)
        filepath = dir_name + "\\" + file
        open_face_data = pd.read_csv(filepath)
        au_scores = open_face_data.loc[:, ["frame", " confidence", " AU01_r", " AU02_r", " AU04_r", " AU05_r", " AU06_r",
                                           " AU07_r", " AU09_r", " AU10_r", " AU12_r", " AU14_r",
                                           " AU15_r", " AU17_r", " AU20_r", " AU23_r", " AU25_r", " AU26_r"]]
        # Only want frames with > 80% confidence
        confident = au_scores[au_scores[" confidence"] > 0.8]
        
        # if there aren't enough samples, the video isn't included in the dataset
        if confident.shape[0] > 35:
            
            #This creates a sample of 35 frames and maintains the time series through sort
            samples = confident.sample(n=35).sort_values(by="frame").drop(columns = ['frame', ' confidence'])
            weights = file.split('_')
            weight = weights[1].rstrip("lbs")
            weight = float(weight)

            # find the weights and accordingly decide the label
            if weight == 7.5:
                if count == 1:
                    y = np.full(35, 0).T
                else :
                    y = np.dstack([y, np.full(35, 0).T])
                Y.append(0)
            elif weight == 25:
                if count == 1:
                    y = np.full(35, 1).T
                else :
                    y = np.dstack([y, np.array(np.full(35, 1).T)])
                Y.append(1)
            else:
                if count == 1:
                    y = np.full(35, 2).T
                else :
                    y = np.dstack([y, np.full(35, 2).T])
                Y.append(2)
                
            if count == 1:
                X = samples.to_numpy()
                
            else:
                print(X.shape)
                X = np.dstack([X, samples.to_numpy()])



In [12]:

# Adjusting the shape of the X set to fit the LSTM
real_X = X.reshape((X.shape[2], 35, 16))

# Adjusting the shape of the y set to fit the LSTM
real_y = y.reshape((y.shape[2], 35, 1))

Y = np.array(Y)

# Generates random numbers to select the train set
r = sample(range(0, real_X.shape[0]), 2516)
X_train = real_X[r,]
Y_train = Y[r,]
X_test = np.delete(real_X, r, 0)
Y_test = np.delete(Y, r, 0)


(3594, 35, 18)


In [ ]:

# 1st LSTM model. 3D input and 3D output; each frame has a lable
model = Sequential()
model.add(LSTM(80, activation='relu', input_shape=(35, 16)))
model.add(Dense(10, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
print(model.summary())
model.fit(real_X, real_y, epochs=2000, validation_split=0.2, batch_size=5)



In [ ]:

# 2nd LSTM Model. 3D input and 1D output. Each Video (batch of 35 frames and 16 features) has a label
model_1 = Sequential()
model_1.add(LSTM(50, activation='relu', input_shape=(35, 16)))
model_1.add(Dense(1))
model_1.compile(optimizer='adam', 
                loss='categorical_crossentropy',
                metrics=['accuracy'])
history = model_1.fit(real_X, Y, epochs=1000, validation_split=0.2, verbose=1)


In [ ]:

# Evaluation of 2nd LSTM Model
print(mean_squared_error(Y_test, model_1.predict(X_test)))
scores = model_1.evaluate(X_test, Y_test)
print(scores[1])
